In [1]:
from model import *
from dilateModel import *
from data import *
from PIL import Image

Using TensorFlow backend.
/srv/conda/envs/notebook/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/srv/conda/envs/notebook/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/srv/conda/envs/notebook/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/srv/conda/envs/notebook/lib/python3.7/site-packag

In [3]:
data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')
myGene = trainGenerator(16,'/data/spacenet/bldg/AllTrain','PAN-PNG','GT-PNG',data_gen_args,save_to_dir = None)
model = unetDil('Humphries_Bragg_Weights.hdf5')
model_checkpoint = ModelCheckpoint('End_Year_Dil_1.hdf5', monitor='loss',verbose=1, save_best_only=True)
model.fit_generator(myGene,steps_per_epoch=2000,epochs=5,callbacks=[model_checkpoint])

Epoch 1/5
Found 10586 images belonging to 1 classes.
Found 10586 images belonging to 1 classes.
2000/2000 [==============================] - 1107s 553ms/step - loss: 0.1124 - accuracy: 0.9585

Epoch 00001: loss improved from inf to 0.11240, saving model to End_Year_Dil_1.hdf5
Epoch 2/5
2000/2000 [==============================] - 1104s 552ms/step - loss: 0.1054 - accuracy: 0.9607

Epoch 00002: loss improved from 0.11240 to 0.10537, saving model to End_Year_Dil_1.hdf5
Epoch 3/5
2000/2000 [==============================] - 1106s 553ms/step - loss: 0.1045 - accuracy: 0.9605

Epoch 00003: loss improved from 0.10537 to 0.10448, saving model to End_Year_Dil_1.hdf5
Epoch 4/5
2000/2000 [==============================] - 1104s 552ms/step - loss: 0.0987 - accuracy: 0.9625

Epoch 00004: loss improved from 0.10448 to 0.09867, saving model to End_Year_Dil_1.hdf5
Epoch 5/5
2000/2000 [==============================] - 1108s 554ms/step - loss: 0.0926 - accuracy: 0.9646

Epoch 00005: loss improved from

In [ ]:
model = unetDil()
model.load_weights("unet_membrane_AllTrain_Dil_1.hdf5")

In [ ]:
def predImgGen(imgPath, target_size=(256,256)):
    img = io.imread(imgPath,as_gray = True)
    img = img / 255
    img = trans.resize(img,target_size)
    img = np.reshape(img,img.shape+(1,))
    img = np.reshape(img,(1,)+img.shape)
    return img

In [ ]:
def predImgGen2(imgPath, target_size=(256,256)):
    img = io.imread(imgPath,as_gray = True)
    #img = img / 255
    img = trans.resize(img,target_size)
    img = np.reshape(img,img.shape+(1,))
    img = np.reshape(img,(1,)+img.shape)
    return img

In [ ]:
test_dir = '/data/spacenet/bldg/AllTest/PAN-PNG'
for i,j,k in os.walk(test_dir):
    for file in k:
        if (os.path.splitext(file)[1] == '.png'):
            PANImgPath = os.path.join(test_dir, file)
            testImg = predImgGen(PANImgPath)
            pred = model.predict(testImg)
        
            predRe = pred.reshape(256,256)
            maskArrThresh = np.zeros((predRe.shape[0], predRe.shape[1], 4)) 
            maskArrThresh[:,:, 0] = 50
            maskArrThresh[:,:, 1] = 255
            maskArrThresh[:,:, 2] = 0
            for row in range(0, predRe.shape[0]):
                for col in range(0, predRe.shape[1]):
                    maskArrThresh[row, col, 3] = 255 - int(predRe[row, col] * 255)
            maskArrThresh = maskArrThresh.astype("uint8")
            maskImg = Image.fromarray(maskArrThresh, 'RGBA')
        
            PANImg = Image.open(PANImgPath)
            PANImgT = PANImg.convert('RGBA')
        
            PANDim = (PANImgT.width, PANImgT.height)
            maskImgRe = maskImg.resize(PANDim) 
        
            comp = Image.alpha_composite(PANImgT, maskImgRe)
            comp.save(file)